# <font color = '#FF003D'> ***Synchronous colorectal cancer-liver metastasis project***

**_______________________________________________________________________________________________________________________________________________________________________________________________________________**

# <font color = '#FF003D'> ***==== CODE 3: Myeloid cell analysis ====***

# Python library

In [ ]:
import os
import math
import warnings
import datetime

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import scipy
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from cycler import cycler
import openpyxl
import scvi

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

import rpy2

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

In [ ]:
result_folder = '.../Analysis/Synchro/'
data_folder = '.../Analysis/Synchro/Data/'

sc.settings.verbosity = 4
warnings.filterwarnings('ignore')
sc.set_figure_params(dpi = 100, dpi_save = 1000, facecolor = 'white')

# R library

In [ ]:
! python -m rpy2.situation

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R 
library(Seurat)
library(SeuratDisk)
library(SeuratData)
library(SeuratWrappers)
library(SeuratObject)

library(openxlsx)
library(ggplot2)
library(ggraph)
library(ggrepel)

library(dplyr)
library(reticulate)
library(patchwork)

library(EnhancedVolcano)

# 1a. **Myeloid cells / Peripheral blood**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Reclustering_Myeloid.h5ad")
Myeloid = sc.read_h5ad(outfilename)
Myeloid_BLOOD = Myeloid[Myeloid.obs['tissue_origin'].isin(['Blood'])].copy()
for lbl in ['leiden', 'res_0.5', 'res_0.8', 'res_1.0', 'res_1.5', 'res_1.8']:
    del Myeloid_BLOOD.obs[lbl]
    del Myeloid_BLOOD.uns
Myeloid_BLOOD    

In [ ]:
# use scVI latent space for UMAP generation
sc.pp.neighbors(Myeloid_BLOOD, n_neighbors = 15, use_rep = "X_scVI", key_added = "scVI")
sc.tl.leiden(Myeloid_BLOOD, key_added = "leiden", resolution = 0.5, neighbors_key = "scVI")
sc.tl.paga(Myeloid_BLOOD, groups = 'leiden', neighbors_key = "scVI")
sc.pl.paga(Myeloid_BLOOD, frameon = True)
sc.tl.umap(Myeloid_BLOOD, init_pos = 'paga', neighbors_key = "scVI")

## UMAP

In [ ]:
# Supplementary figure 3A
Myeloid_BLOOD.obs['Myeloid_site_short'] = "B" + Myeloid_BLOOD.obs['leiden'].astype(str)
colors = {
    'B0': '#d60000',
    'B1': '#8c3bff',
    'B2': '#018700',
    'B3': '#00acc6',
    'B4': '#97ff00',
    'B5': '#ff7ed1',
    'B6': '#6b004f'}

sc.pl.umap(Myeloid_BLOOD, color = 'Myeloid_site_short',
           legend_loc = 'on data', legend_fontsize = 14, legend_fontoutline = 2, legend_fontweight = 1, frameon = True, palette = colors,
           save = 'Myeloid_BLOOD – Myeloid_site_short.png'

## DotPlot

In [ ]:
# Supplementary figure 3B
Annotation = ['CD14', 'S100A8', 'S100A9', 'RBP7', 'FOSB', 
         'HLA-DPB1', 'HLA-DPA1', 'HLA-DRB1', 'HLA-DRA',
         'PPBP', 'PF4', 'CAVIN2', 'TUBB1',
         'NFKB1', 'NRF1', 'TREM1', 'TLR4',
         'AHSP', 'ALAS2', 'CA1',
         'IL7R', 'GZMA', 'GZMB', 'PRF1', 'GNLY', 'NKG7', 'CTSW', 'CD2',
         'FCGR3A', 'RHOC', 'CDKN1C', 'MS4A7',
         'XBP1', 'MZB1', 'CLEC4C', 'IL3RA', 'IRF7']

sc.pl.dotplot(Myeloid_BLOOD, Annotation, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False,
              save = 'Myeloid_BLOOD annotation.png')

# 1b. **Myeloid cells / Colon**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Reclustering_Myeloid.h5ad")
Myeloid = sc.read_h5ad(outfilename)   
Myeloid_COLON = Myeloid[Myeloid.obs['tissue_origin'].isin(['Colon'])].copy()
for lbl in ['leiden', 'res_0.5', 'res_0.8', 'res_1.0', 'res_1.5', 'res_1.8']:
    del Myeloid_COLON.obs[lbl]
    del Myeloid_COLON.uns
Myeloid_COLON

In [ ]:
# use scVI latent space for UMAP generation
sc.pp.neighbors(Myeloid_COLON, n_neighbors = 15, use_rep = "X_scVI", key_added = "scVI")
sc.tl.leiden(Myeloid_COLON, key_added = "leiden", resolution = 0.5, neighbors_key = "scVI")
sc.tl.paga(Myeloid_COLON, groups = 'leiden', neighbors_key = "scVI")
sc.pl.paga(Myeloid_COLON, frameon = True)
sc.tl.umap(Myeloid_COLON, init_pos = 'paga', neighbors_key = "scVI")

## UMAP

In [ ]:
# Figure 2A
Myeloid_COLON.obs['Myeloid_site_short'] = "C" + Myeloid_COLON.obs['leiden'].astype(str)
colors = {
    'C0': '#ffa52f',
    'C1': '#573b00',
    'C2': '#005659',
    'C3': '#0000dd',
    'C4': '#00fdcf',
    'C5': '#a17569',
    'C6': '#bcb6ff',
    'C7': '#95b577',
    'C8': '#bf03b8',
    'C9': '#645474'}

sc.pl.umap(Myeloid_COLON, color = 'Myeloid_site_short',
           legend_loc = 'on data', legend_fontsize = 14, legend_fontoutline = 2, legend_fontweight = 1, frameon = True, palette = colors,
           save = 'Myeloid_COLON – Myeloid_site_short.png'

## DotPlot

In [ ]:
# Figure 2B
Genes_Colon = ["C5AR1", "CD68", "LYZ",
               'S100A8', 'S100A9', "S100A12",  'CCL3L1', "FCN1",
               'CD163', 'MRC1', 'C1QA', 'C1QB', 'C1QC', "SLC40A1", "FRMD4A", "SLCO2B1",
               'LAMP3', 'KIF2A', 'RPS27L', 'GRSF1', 'CCL19', 'CCR7',
               'XBP1', 'MZB1', 'CLEC4C', 'IL3RA', 'IRF7', 'CXCR3']
order = ['C0', 'C2', 'C4', 'C6', 'C3', 'C5', 'C8', 'C9', 'C1', 'C7']  

sc.pl.dotplot(Myeloid_COLON, Genes_Colon, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False, categories_order = order, 
              save = 'Myeloid_COLON annotation.png')

In [ ]:
# Figure 2F
ImmuneCheck = ['PDCD1', 'CTLA4', 'LAG3', 'HAVCR2', 'TIGIT', 'VSIR', 'IDO1', 'ICOS', 'CD96', 'SIGIRR', 'CD274', 'NECTIN2'] 
order = ['C0', 'C2', 'C4', 'C6', 'C3', 'C5', 'C8']  

subset_macro = Myeloid_COLON[Myeloid_COLON.obs['Myeloid_site_short'].isin(order)].copy() #To remove DCs
sc.pl.dotplot(subset_macro, ImmuneCheck, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False, categories_order = order, dot_max = 1,
              save = 'Myeloid_COLON ImmunoCheck.png')

## Cell frequency

In [ ]:
# Figure 2C
## Outer circle
Cells = pd.crosstab([Myeloid_COLON.obs['Myeloid_site_short']], [Myeloid_COLON.obs['tissue_origin']], dropna = True)
Cells.to_excel('.../Analysis/Synchro/figures/Cells Myeloid_COLON – Myeloid_site_short.xlsx', index = True)

## Inner circle
Myeloid_COLON.obs['CellFamily'] = Myeloid_COLON.obs['Myeloid_site_short'].map({
    'C0': 'Inflammatory TAMs',
    'C1': 'Others',
    'C2': 'Inflammatory TAMs',
    'C3': 'Immunoregulatory TAMs',
    'C4': 'Inflammatory TAMs',
    'C5': 'Immunoregulatory TAMs',
    'C6': 'Inflammatory TAMs',
    'C7': 'Others',
    'C8': 'Immunoregulatory TAMs',
    'C9': 'Others'})
Cells = pd.crosstab([Myeloid_COLON.obs['CellFamily']], [Myeloid_COLON.obs['tissue_origin']], dropna = True)
Cells.to_excel('.../Analysis/Synchro/figures/Cells Myeloid_COLON – CellFamily.xlsx', index = True)

## Heatmap top10 DEGs/cluster

In [ ]:
# Create the object to convert to Seurat object for DEGs
Myeloid_COLON_raw = sc.AnnData(X = Myeloid_COLON.layers['raw_counts'],
                     obs = Myeloid_COLON.obs[['batch_id', 'tissue_origin', 'patient_ID', 'Myeloid_site_short']],
                     var = Myeloid_COLON.var[['gene_ids']],
                     obsm = Myeloid_COLON.obsm)

outfilename = os.path.join(data_folder, "Myeloid_COLON_raw.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid_COLON_raw.write(outfilename)
print("Done!")

# Export metadata
Myeloid_COLON_raw.obs.to_excel('.../Analysis/Synchro/figures/Myeloid_COLON_raw_metadata.xlsx')

In [ ]:
%%R
Convert(".../Analysis/Synchro/Data/Myeloid_COLON_raw.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
Myeloid_COLON_raw = LoadH5Seurat(".../Analysis/Synchro/Data/Myeloid_COLON_raw.h5seurat", meta.data = F)
meta = read.xlsx(".../Analysis/Synchro/figures/Myeloid_COLON_raw_metadata.xlsx", rowNames = T)
Myeloid_COLON_raw@meta.data = meta
saveRDS(Myeloid_COLON_raw, ".../Analysis/Synchro/Data/Myeloid_COLON_raw.RDS")

In [ ]:
%%R
Idents(Myeloid_COLON_raw) = 'Myeloid_site_short'
Myeloid_COLON_raw = subset(Myeloid_COLON_raw, idents = c('C0', 'C2', 'C3', 'C4', 'C5', 'C6', 'C8')) # subset of TAMs
DEGs = FindAllMarkers(Myeloid_COLON_raw, logfc.threshold = 0, test.use = "negbinom", latent_vars = 'patient_ID')
write.xlsx(DEGs, "DEGs - Myeloid_COLON_raw all vs all – negbinom.xlsx", rowNames = T)

In [ ]:
# Figure 2D
Subset = Myeloid_COLON[~Myeloid_COLON.obs['Myeloid_site_short'].isin(['C1', 'C7', 'C9'])].copy()
Subset.obs['Myeloid_site_short'] = Subset.obs['Myeloid_site_short'].cat.reorder_categories(['C0', 'C2', 'C4', 'C6', 'C3', 'C5', 'C8'])
Subset.layers["scaled"] = sc.pp.scale(Subset, copy = True).X

top10_colon = ["S100A8", "CCL4L2", "CCL3L1", "CCL3", "C15orf48", "S100A9", "CCL4", "CCL20", "CXCL10", "MT2A",
               "IFITM3", "TIMP1", "S100A10", "HLA-DQB1", "CRIP1", "HLA-DQA1", "HLA-DRB1", "IFITM1", "HLA-DPA1", "IFI30",
               "DOCK4", "FNDC3B", "MAP4K4", "ZEB2", "ATP13A3", "PTPRJ", "CD44", "MAML2", "ARHGAP26", "NRIP1",
               "S100A9", "LUCAT1", "CXCL8", "S100A12", "MCTP2", "FCN1", "AQP9", "VCAN", "CD52", "SLC11A1",
               "AREG", "IL1R2", "FOS", "NR4A2", "ATF3", "EGR1", "FOSB", "NR4A1", "JUNB", "JUN",
               "RNASE1", "C1QA", "C1QB", "LYVE1", "F13A1", "PLTP", "RBPJ", "CD163", "PDK4", "FOLR2",
               "CCL2", "HMOX1", "ANKH", "MITF", "PPARG", "CTSL", "SPRED1", "NRP1", "SLC4A7", "ME1"]

sc.pl.matrixplot(Subset, top10_colon, groupby = 'Myeloid_site_short', swap_axes = True,
                 layer = "scaled", vmin = -2, vmax = 2, cmap = 'RdBu_r',
                 save = "Myeloid_COLON heatmap top 10 DEGs_Only Macrophages_Scaled.png")

## Module scores

In [ ]:
# Figure 2E input data
APC_TAMs = ["HLA-DRA", "HLA-DRB1", "HLA-DRB2", "HLA-DRB3", "HLA-DRB4", "HLA-DRB5", "HLA-DRB6", "HLA-DRB7", "HLA-DRB8", "HLA-DRB9", "HLA-DQA1", "HLA-DQB1", "HLA-DQA2", "HLA-DQB2", "HLA-DQB3", "HLA-DOA", "HLA-DOB", "HLA-DMA", "HLA-DMB", "HLA-DPA1", "HLA-DPB1", "HLA-DPA2", "HLA-DPB2", "HLA-DPA3"]
LA_TAMs = ['ACP5', 'APOE', 'APOC1', 'C1QA', 'C1QB', 'C1QC', 'CCL18', 'CD163', 'CD36', 'CD63', 'CTSB', 'CTSD', 'CTSL', 'F13A1', 'FABP5', 'FOLR2', 'GPNMB', 'LIPA', 'MARCO', 'MERTK',  'MMP9', 'MRC1', 'NRF1', 'PLA2G7', 'RNASE1', 'TFDP2', 'ZEB1', 'HIF1A', 'MAF', 'MAFB', 'TCF4', 'TFEC']
Angio_TAMs = ['CCL20', 'CD163', 'CD300E', 'CD44', 'CD55', 'CEBPB', 'CTSB', 'EREG', 'FCN1', 'FLT1',  'IL1B', 'IL1RN', 'CXCL8', 'MIF', 'OLR1', 'PPARG', 'S100A8', 'S100A9', 'S100A12', 'SERPINB2', 'THBS1', 'TIMP1', 'VCAN', 'VEGFA', 'BACH1', 'CEBPB', 'FOSL2', 'HIF1A', 'NFKB1', 'RUNX1', 'ZEB2']

Subset = Myeloid_COLON[~Myeloid_COLON.obs['Myeloid_site_short'].isin(['C1', 'C7', 'C9'])].copy()

sc.tl.score_genes(Subset, LA_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'LA_TAMs', random_state = 0, copy = False)
sc.tl.score_genes(Subset, Angio_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'Angio_TAMs', random_state = 0, copy = False)
sc.tl.score_genes(Subset, APC_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'APC_TAMs', random_state = 0, copy = False)

Scores = Subset.obs[['Myeloid_site_short', 'LA_TAMs', 'Angio_TAMs', 'APC_TAMs']]

scaler = MinMaxScaler()
scores_scaled = scaler.fit_transform(Scores)
scores_scaled = pd.DataFrame(scores_scaled, columns = ['LA_TAMs_scaled', 'Angio_TAMs_scaled', 'APC_TAMs_scaled'], index = Subset.obs_names)
Scores = pd.concat([Subset.obs[['Myeloid_site_short']], Scores, scores_scaled], axis = 1)
Score.to_excel('figures/Myeloid_COLON – Module scores x Myeloid_site_short.xlsx', index = True)

In [ ]:
%%R
# Figure 2E Radar Chart
library(dplyr)

setwd(".../figures")
path = ".../figures"

RadarPlot = read.xlsx("Myeloid_COLON – Module scores x Myeloid_site_short.xlsx", rowNames = T)

LA_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, LA_TAMs_scaled)

Angio_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, Angio_TAMs_scaled)

APC_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, APC_TAMs_scaled)

Colors_scaled = c('Angio_TAMs' = '#008080', 'LA_TAMs' = '#a2798f', 'APC_TAMs' = '#9e379f')
colors_in = alpha(Colors_scaled, 0.8)

png(file = file.path(path, 'Angio_TAMs_scaled COLON.png'), units = "in", width = 6, height = 6, res = 1000, bg = "white")

radarchart(Angio_TAMs_scaled, 
            axistype = 0, axislabcol = 'grey10', 
            pcol = 'black', pfcol = alpha('#9e379f', 0.8), plwd = 1, plty = 1, pty = 32,
            cglcol = "grey50", cglty = 1,  cglwd = 1, 
            vlcex = 1)
legend(x = 1, y = 1, legend = rownames(Angio_TAMs_scaled[-c(1,2),]), bty = "n", pch = 20 , col = alpha('#9e379f', 0.8) , text.col = "grey", cex = 1.2, pt.cex = 3)

dev.off()

In [ ]:
# Supplementary figure XX
Cytoxic_Inflammatory = ['PRF1', 'GZMM', 'GZMA', 'KLRB1', 'GZMK', 'GZMH', 'IL32', 'CCL5', 'GZMB', 'GNLY', 'IFNG', 'TNF', 'CST7', 'NKG7', 'EOMES', 'EFHD2', 'F2R', 'SLAMF7', "IL1B", "IL2", "IL4"]
Subset = Myeloid_COLON[~Myeloid_COLON.obs['Myeloid_site_short'].isin(['C1', 'C7', 'C9'])].copy()

sc.tl.score_genes(Subset, Cytoxic_Inflammatory, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'Cytoxic_Inflammatory_score', random_state = 0, copy = False)
sc.pl.umap(Subset, color = 'Cytoxic_Inflammatory_score', vmin = 0, vmax = 1, cmap = 'inferno', title = 'Cytotoxic-Inflammatory Score', save = 'Myeloid_COLON – Cytoxic_Inflammatory_Score.png')

# 1c. **Myeloid cells / Liver**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Reclustering_Myeloid.h5ad")
Myeloid = sc.read_h5ad(outfilename)   
Myeloid_LIVER = Myeloid[Myeloid.obs['tissue_origin'].isin(['Liver'])].copy()
for lbl in ['leiden', 'res_0.5', 'res_0.8', 'res_1.0', 'res_1.5', 'res_1.8']:
    del Myeloid_LIVER.obs[lbl]
    del Myeloid_LIVER.uns
Myeloid_LIVER

In [ ]:
# use scVI latent space for UMAP generation
sc.pp.neighbors(Myeloid_LIVER, n_neighbors = 15, use_rep = "X_scVI", key_added = "scVI")
sc.tl.leiden(Myeloid_LIVER, key_added = "leiden", resolution = 0.5, neighbors_key = "scVI")
sc.tl.paga(Myeloid_LIVER, groups = 'leiden', neighbors_key = "scVI")
sc.pl.paga(Myeloid_LIVER, frameon = True)
sc.tl.umap(Myeloid_LIVER, init_pos = 'paga', neighbors_key = "scVI")

## UMAP

In [ ]:
# Figure 3A
Myeloid_LIVER.obs['Myeloid_site_short'] = "L" + Myeloid_LIVER.obs['leiden'].astype(str)
colors = {
    'L0': '#790000',
    'L1': '#0774d8',
    'L2': '#fdf490',
    'L3': '#004b00',
    'L4': '#8e7900',
    'L5': '#ff7266',
    'L6': '#edb8b8',
    'L7': '#5d7e66',
    'L8': '#9ae4ff',
    'L9': '#eb0077',
    'L10': '#a57bb8',
    'L11': '#5900a3'}

sc.pl.umap(Myeloid_LIVER, color = 'Myeloid_site_short',
           legend_loc = 'on data', legend_fontsize = 14, legend_fontoutline = 2, legend_fontweight = 1, frameon = True, palette = colors,
           save = 'Myeloid_LIVER – Myeloid_site_short.png'

## DotPlot

In [ ]:
# Figure 3B
Genes_Liver = ["C5AR1", "CD68", "LYZ",
               'VCAN', 'S100A8', 'S100A9', "FCN1", 'CFD', 'CTSL', 'MMP19', 'MARCO',
               'CD163', 'MRC1', 'C1QA', 'C1QB', 'C1QC',  "FRMD4A", "SLCO2B1", "SLC40A1", 'LGMN',
               'IL7R', 'GZMA', 'GZMB', 'PRF1', 'GNLY', 'NKG7', 'CTSW', 'CD2',
               'CADM1', 'CLEC9A', 'XCR1',
               'LAMP3', 'KIF2A', 'RPS27L', 'GRSF1', 'CCL19', 'CCR7',
               'XBP1', 'MZB1', 'CLEC4C', 'IL3RA', 'IRF7']
order = ['L3', 'L8', 'L4', 'L6', 'L0', 'L1', 'L2', 'L5', 'L7', 'L10', 'L11', 'L9']

sc.pl.dotplot(Myeloid_LIVER, Genes_Liver, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False, categories_order = order,
              save = 'Myeloid_LIVER annotation.png')

In [ ]:
# Figure 3G
ImmuneCheck = ['PDCD1', 'CTLA4', 'LAG3', 'HAVCR2', 'TIGIT', 'VSIR', 'IDO1', 'ICOS', 'CD96', 'SIGIRR', 'CD274', 'NECTIN2'] 
order = ['L3', 'L8', 'L4', 'L6', 'L0', 'L1', 'L2', 'L5', 'L7']

subset_macro = Myeloid_LIVER[Myeloid_LIVER.obs['Myeloid_site_short'].isin(order)].copy() #To remove DCs
sc.pl.dotplot(subset_macro, ImmuneCheck, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False, categories_order = order, dot_max = 1,
              save = 'Myeloid_LIVER ImmunoCheck.png')

## Cell frequency

In [ ]:
# Figure 2C
## Outer circle
Cells = pd.crosstab([Myeloid_LIVER.obs['Myeloid_site_short']], [Myeloid_LIVER.obs['tissue_origin']], dropna = True)
Cells.to_excel('figures/Cells Myeloid_LIVER – Myeloid_site_short.xlsx', index = True)

## Inner circle
Myeloid_LIVER.obs['CellFamily'] = Myeloid_LIVER.obs['Myeloid_site_short'].map({
    'L0': 'Immunoregulatory TAMs',
    'L1': 'Immunoregulatory TAMs',
    'L2': 'Immunoregulatory TAMs',
    'L3': 'Inflammatory TAMs',
    'L4': 'Inflammatory TAMs',
    'L5': 'Immunoregulatory TAMs',
    'L6': 'Immunoregulatory TAMs',
    'L7': 'Immunoregulatory TAMs',
    'L8': 'Inflammatory TAMs',
    'L9': 'Others',
    'L10': 'Others',
    'L11': 'Others'})

Cells = pd.crosstab([Myeloid_LIVER.obs['CellFamily']], [Myeloid_LIVER.obs['tissue_origin']], dropna = True)
Cells.to_excel('figures/Cells Myeloid_LIVER – CellFamily.xlsx', index = True)

## Heatmap top10 DEGs/cluster

In [ ]:
# Create the object to convert to Seurat object for DEGs
Myeloid_LIVER_raw = sc.AnnData(X = Myeloid_LIVER.layers['raw_counts'],
                     obs = Myeloid_LIVER.obs[['batch_id', 'tissue_origin', 'patient_ID', 'Myeloid_site_short']],
                     var = Myeloid_LIVER.var[['gene_ids']],
                     obsm = Myeloid_LIVER.obsm)

outfilename = os.path.join(data_folder, "Myeloid_LIVER_raw.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid_LIVER_raw.write(outfilename)
print("Done!")

# Export metadata
Myeloid_LIVER_raw.obs.to_excel('.../Analysis/Synchro/figures/Myeloid_LIVER_metadata.xlsx')

In [ ]:
%%R
Convert(".../Analysis/Synchro/Data/Myeloid_LIVER_raw.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
Myeloid_LIVER_raw = LoadH5Seurat(".../Analysis/Synchro/Data/Myeloid_LIVER_raw.h5seurat", meta.data = F)
meta = read.xlsx(".../Analysis/Synchro/figures/Myeloid_LIVER_metadata.xlsx", rowNames = T)
Myeloid_LIVER_raw@meta.data = meta
saveRDS(Myeloid_LIVER_raw, ".../Analysis/Synchro/Data/Myeloid_LIVER_raw.RDS")

In [ ]:
%%R
Idents(Myeloid_LIVER_raw) = 'Myeloid_site_short'
Myeloid_LIVER_raw = subset(Myeloid_LIVER_raw, idents = c('L3', 'L8', 'L4', 'L6', 'L0', 'L1', 'L2', 'L5', 'L7')) # subset of TAMs
DEGs = FindAllMarkers(Myeloid_LIVER_raw, logfc.threshold = 0, test.use = "negbinom", latent_vars = 'patient_ID')
write.xlsx(DEGs, "DEGs - Myeloid_LIVER_raw all vs all – negbinom.xlsx", rowNames = T)

In [ ]:
# Figure 3D
Subset = Myeloid_LIVER[~Myeloid_LIVER.obs['Myeloid_site_short'].isin(['L9', 'L10', 'L11'])].copy()
Subset.obs['Myeloid_site_short'] = Subset.obs['Myeloid_site_short'].cat.reorder_categories(['L3', 'L8', 'L4', 'L6', 'L0', 'L1', 'L2', 'L5', 'L7'])
Subset.layers["scaled"] = sc.pp.scale(Subset, copy = True).X

top10_liver = ["S100A8", "S100A9", "S100A12", "FNDC3B", "VCAN", "ANPEP", "ZEB2", "NRIP1", "AQP9", "ATP2B1",
              "SMIM25", "FCN1", "FCGR3A", "LST1", "G0S2", "SERPINA1", "LILRA5", "CFD", "LILRB2", "CTSC",
              "CCL5", "CD69", "NKG7", "XCL2", "GNLY", "IL7R", "CEMIP2", "IL32", "RORA", "CD7",
              "APOC1", "APOE", "CTSD", "LYZ", "C5AR1", "PSAP", "THBS1", "FCN1", "FTL", "EREG",
              "IL7R", "CD69", "CCL5", "SYTL3", "ZFP36L2", "RORA", "NKG7", "IFITM1", "KLRB1", "CST7",
              "TOB1", "HLA-DQA1", "TUBA1A", "HLA-DPB1", "AREG", "VIM", "IL1R2", "HLA-DQB1", "HLA-DPA1", "HLA-DRA",
              "HSPA6", "CCL4L2", "HSPB1", "BAG3", "HSPA1B", "CCL3", "HSPA1A", "EGR1", "CCL3L1", "IL1B",
              "CCL4L2", "C1QC", "C1QB", "SELENOP", "TEX14", "CCL3", "C1QA", "LRRC23", "IER3", "PDK4",
              "SPATS2L", "FMNL2", "UST", "FNIP2", "CCSER1", "CHST11", "ALCAM", "SLC8A1", "ATF6", "RAPGEF1"]

sc.pl.matrixplot(Subset, top10_liver, groupby = 'Myeloid_site_short', swap_axes = True,
                 layer = "scaled", vmin = -2, vmax = 2, cmap = 'RdBu_r',
                 save = "Myeloid_LIVER heatmap top 10 DEGs_Only Macrophages_Scaled.png")

## Module scores

In [ ]:
# Figure 3E input data
APC_TAMs = ["HLA-DRA", "HLA-DRB1", "HLA-DRB2", "HLA-DRB3", "HLA-DRB4", "HLA-DRB5", "HLA-DRB6", "HLA-DRB7", "HLA-DRB8", "HLA-DRB9", "HLA-DQA1", "HLA-DQB1", "HLA-DQA2", "HLA-DQB2", "HLA-DQB3", "HLA-DOA", "HLA-DOB", "HLA-DMA", "HLA-DMB", "HLA-DPA1", "HLA-DPB1", "HLA-DPA2", "HLA-DPB2", "HLA-DPA3"]
LA_TAMs = ['ACP5', 'APOE', 'APOC1', 'C1QA', 'C1QB', 'C1QC', 'CCL18', 'CD163', 'CD36', 'CD63', 'CTSB', 'CTSD', 'CTSL', 'F13A1', 'FABP5', 'FOLR2', 'GPNMB', 'LIPA', 'MARCO', 'MERTK',  'MMP9', 'MRC1', 'NRF1', 'PLA2G7', 'RNASE1', 'TFDP2', 'ZEB1', 'HIF1A', 'MAF', 'MAFB', 'TCF4', 'TFEC']
Angio_TAMs = ['CCL20', 'CD163', 'CD300E', 'CD44', 'CD55', 'CEBPB', 'CTSB', 'EREG', 'FCN1', 'FLT1',  'IL1B', 'IL1RN', 'CXCL8', 'MIF', 'OLR1', 'PPARG', 'S100A8', 'S100A9', 'S100A12', 'SERPINB2', 'THBS1', 'TIMP1', 'VCAN', 'VEGFA', 'BACH1', 'CEBPB', 'FOSL2', 'HIF1A', 'NFKB1', 'RUNX1', 'ZEB2']

Subset = Myeloid_LIVER[~Myeloid_LIVER.obs['Myeloid_site_short'].isin(['L9', 'L10', 'L11'])].copy()

sc.tl.score_genes(Subset, LA_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'LA_TAMs', random_state = 0, copy = False)
sc.tl.score_genes(Subset, Angio_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'Angio_TAMs', random_state = 0, copy = False)
sc.tl.score_genes(Subset, APC_TAMs, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'APC_TAMs', random_state = 0, copy = False)

Scores = Subset.obs[['Myeloid_site_short', 'LA_TAMs', 'Angio_TAMs', 'APC_TAMs']]

scaler = MinMaxScaler()
scores_scaled = scaler.fit_transform(Scores)
scores_scaled = pd.DataFrame(scores_scaled, columns = ['LA_TAMs_scaled', 'Angio_TAMs_scaled', 'APC_TAMs_scaled'], index = Subset.obs_names)
Scores = pd.concat([Subset.obs[['Myeloid_site_short']], Scores, scores_scaled], axis = 1)
Score.to_excel('figures/Myeloid_LIVER – Module scores x Myeloid_site_short.xlsx', index = True)

In [ ]:
%%R
# Figure 3E Radar Chart
library(dplyr)

setwd(".../figures")
path = ".../figures"

RadarPlot = read.xlsx("Myeloid_LIVER – Module scores x Myeloid_site_short.xlsx", rowNames = T)

LA_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, LA_TAMs_scaled)

Angio_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, Angio_TAMs_scaled)

APC_TAMs_scaled = RadarPlot %>%
    select(Myeloid_site_short, APC_TAMs_scaled)

Colors_scaled = c('Angio_TAMs' = '#008080', 'LA_TAMs' = '#a2798f', 'APC_TAMs' = '#9e379f')
colors_in = alpha(Colors_scaled, 0.8)

png(file = file.path(path, 'Angio_TAMs_scaled LIVER.png'), units = "in", width = 6, height = 6, res = 1000, bg = "white")

radarchart(Angio_TAMs_scaled, 
            axistype = 0, axislabcol = 'grey10', 
            pcol = 'black', pfcol = alpha('#9e379f', 0.8), plwd = 1, plty = 1, pty = 32,
            cglcol = "grey50", cglty = 1,  cglwd = 1, 
            vlcex = 1)
legend(x = 1, y = 1, legend = rownames(Angio_TAMs_scaled[-c(1,2),]), bty = "n", pch = 20 , col = alpha('#9e379f', 0.8) , text.col = "grey", cex = 1.2, pt.cex = 3)

dev.off()

In [ ]:
# Figure 3F
Cytoxic_Inflammatory = ['PRF1', 'GZMM', 'GZMA', 'KLRB1', 'GZMK', 'GZMH', 'IL32', 'CCL5', 'GZMB', 'GNLY', 'IFNG', 'TNF', 'CST7', 'NKG7', 'EOMES', 'EFHD2', 'F2R', 'SLAMF7', "IL1B", "IL2", "IL4"]
Subset = Myeloid_LIVER[~Myeloid_LIVER.obs['Myeloid_site_short'].isin(['L9', 'L10', 'L11'])].copy()

sc.tl.score_genes(Subset, Cytoxic_Inflammatory, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'Cytoxic_Inflammatory_score', random_state = 0, copy = False)
sc.pl.umap(Subset, color = 'Cytoxic_Inflammatory_score', vmin = 0, vmax = 1, cmap = 'inferno', title = 'Cytotoxic-Inflammatory Score', save = 'Myeloid_LIVER – Cytoxic_Inflammatory_Score.png')

## ViolinPlot

In [ ]:
# Supplementary figure 2A
KC = ['CD5L', 'VSIG4', 'MS4A4A', 'SLC16A9', 'TMEM132E', 'SLC1A3', 'CD163', 'FOLR2', 'TIMD4', 'GFRA2', 'ADRB1', 'TMEM26', 'SLC40A1',
      'VCAM1', 'SUCNR1', 'NDST3', 'CETP', 'LYVE1', 'MARCO', 'SDC3', 'CXCL12', 'LILRB5', 'SCD', 'SELENBP1',
      'MMP19', 'LGMN', 'HMOX1', 'CTSL', 'C2', 'FABP3', 'TMEM37', 'RND3']

colors_liver = list(Myeloid_LIVER.uns['Myeloid_site_short_colors'])

sc.pl.stacked_violin(Myeloid_LIVER, KC, 'Myeloid_site_short', standard_scale = 'var', swap_axes = False, dendrogram = False, scale = 'count', row_palette = colors_liver, yticklabels = True,
                     save = "Myeloid_LIVER KC signature.png")

# 2. **Myeloid cells / Total object**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Reclustering_Myeloid.h5ad")
Myeloid = sc.read_h5ad(outfilename)   

Myeloid.obs['Myeloid_site_short'] = ""
Myeloid_BLOOD.obs['Myeloid_site_short'] = Myeloid_BLOOD.obs['Myeloid_site_short'].astype(str)
Myeloid_COLON.obs['Myeloid_site_short'] = Myeloid_COLON.obs['Myeloid_site_short'].astype(str)
Myeloid_LIVER.obs['Myeloid_site_short'] = Myeloid_LIVER.obs['Myeloid_site_short'].astype(str)
Myeloid.obs['Myeloid_site_short'].update(Myeloid_BLOOD.obs['Myeloid_site_short'])
Myeloid.obs['Myeloid_site_short'].update(Myeloid_COLON.obs['Myeloid_site_short'])
Myeloid.obs['Myeloid_site_short'].update(Myeloid_LIVER.obs['Myeloid_site_short'])

Myeloid.obs['CellFamily'] = Myeloid.obs['Myeloid_site_short'].map({
    'B0': 'cMono_CD14',
    'B1': 'cMono_CD14',
    'B2': 'cMono_CD14',
    'B3': 'cMono_CD14',
    'B4': 'cMono_CD14',
    'B5': 'ncMono_FCGR3A',
    'B6': 'pDCs',
    'C0': 'Inflammatory TAMs',
    'C1': 'pDCs',
    'C2': 'Inflammatory TAMs',
    'C3': 'Immunoregulatory TAMs',
    'C4': 'Inflammatory TAMs',
    'C5': 'Immunoregulatory TAMs',
    'C6': 'Inflammatory TAMs',
    'C7': 'pDCs',
    'C8': 'Immunoregulatory TAMs',
    'C9': 'DC_LAMP3',
    'L0': 'Immunoregulatory TAMs',
    'L1': 'Immunoregulatory TAMs',
    'L2': 'Immunoregulatory TAMs',
    'L3': 'Inflammatory TAMs',
    'L4': 'Inflammatory TAMs',
    'L5': 'Immunoregulatory TAMs',
    'L6': 'Immunoregulatory TAMs',
    'L7': 'Immunoregulatory TAMs',
    'L8': 'Inflammatory TAMs',
    'L9': 'pDCs',
    'L10': 'cDC1',
    'L11': 'DC_LAMP3'})

colors_Myeloid_site_short = {
    'B0': '#d60000',
    'B1': '#8c3bff',
    'B2': '#018700',
    'B3': '#00acc6',
    'B4': '#97ff00',
    'B5': '#ff7ed1',
    'B6': '#6b004f',
    'C0': '#ffa52f',
    'C1': '#573b00',
    'C2': '#005659',
    'C3': '#0000dd',
    'C4': '#00fdcf',
    'C5': '#a17569',
    'C6': '#bcb6ff',
    'C7': '#95b577',
    'C8': '#bf03b8',
    'C9': '#645474',
    'L0': '#790000',
    'L1': '#0774d8',
    'L2': '#fdf490',
    'L3': '#004b00',
    'L4': '#8e7900',
    'L5': '#ff7266',
    'L6': '#edb8b8',
    'L7': '#5d7e66',
    'L8': '#9ae4ff',
    'L9': '#eb0077',
    'L10': '#a57bb8',
    'L11': '#5900a3'}

colors_CellFamily = {
    'cMono_CD14': '#7a49a5',
    'ncMono_FCGR3A': '#0000ff',
    'Inflammatory TAMs': '#0b5987',
    'Immunoregulatory TAMs': '#870B59',
    'cDC1': '#638CFF',
    'DC_LAMP3': '#ffd663',
    'pDCs': '#03989e'}

Myeloid

## UMAP

In [ ]:
# Figure 1B
sc.pl.umap(Myeloid, color = 'tissue_origin', save = 'Myeloid total – tissue_origin.png'

In [ ]:
# Figure 1C
sc.pl.umap(Myeloid, color = 'CellFamily', palette = colors_CellFamily, save = 'Myeloid total – CellFamily.png'

In [ ]:
# Supplementary figure 4A left
colors_Myeloid_site_short_refined = {
    'B0': '#d60000',
    'B1': '#8c3bff',
    'B2': '#018700',
    'B3': '#00acc6',
    'B4': '#97ff00',
    'B5': '#ff7ed1',
    'B6': '#bcbcbc',
    'C0': '#ffa52f',
    'C1': '#bcbcbc',
    'C2': '#005659',
    'C3': '#0000dd',
    'C4': '#00fdcf',
    'C5': '#a17569',
    'C6': '#bcb6ff',
    'C7': '#bcbcbc',
    'C8': '#bf03b8',
    'C9': '#bcbcbc',
    'L0': '#790000',
    'L1': '#0774d8',
    'L2': '#fdf490',
    'L3': '#004b00',
    'L4': '#8e7900',
    'L5': '#ff7266',
    'L6': '#edb8b8',
    'L7': '#5d7e66',
    'L8': '#9ae4ff',
    'L9': '#bcbcbc',
    'L10': '#bcbcbc',
    'L11': '#bcbcbc'}

sc.pl.umap(Myeloid, color = 'Myeloid_site_short',
           legend_loc = 'on data', legend_fontsize = 14, legend_fontoutline = 2, legend_fontweight = 1, frameon = True, palette = colors_Myeloid_site_short_refined,
           save = 'Myeloid total – Myeloid_site_short with DCs in gray.png'

## DotPlot

In [ ]:
# Figure 1C
Signature = ['CD14', 'FCGR3A', 'CD68', 'LYZ', 'VCAN', 'S100A8', 'S100A9', 'FCN1', 'CFD', 'CTSL', 'CD163', 'MRC1', 'C1QA', 'C1QB', 'C1QC',
            'CADM1', 'CLEC9A', 'XCR1', 'LAMP3', 'KIF2A', 'RPS27L', 'GRSF1', 'CCL19', 'CLEC4C', 'IL3RA', 'IRF7']
order = ['cMono_CD14', 'ncMono_FCGR3A', 'Inflammatory TAMs', 'Immunoregulatory TAMs', 'cDC1', 'DC_LAMP3', 'pDCs']

sc.pl.dotplot(Myeloid, Signature, 'CellFamily', standard_scale = 'var', swap_axes = False, dendrogram = False, dot_max = 1, categories_order = order,
              save = 'Myeloid annotation.png')

## Cell frequency

In [ ]:
# Figure 2D input data
Cells = pd.crosstab([Myeloid.obs['CellFamily']], [Myeloid.obs['tissue_origin']], dropna = True)
Cells.to_excel('figures/Cells Myeloid – CellFamily x tissue_origin.xlsx', index = True)

In [ ]:
%%R
# Figure 2D Alluvial Plot
library(ggalluvial)
Fam = read.xlsx('.../figures/Cells Myeloid – CellFamily x tissue_origin.xlsx', sheet = 'Alluvion')

col = c('Inflammatory TAMs' = '#0b5987',
        'Immunoregulatory TAMs' = '#870B59',
        'pDCs' = '#03989e',
        'DC_LAMP3' = '#ffd663',
        'cDC1' = '#638CFF',
        'cMono_CD14' = '#7a49a5',
        'ncMono_FCGR3A' = '#0000ff')

png(file = file.path(".../figures/", "Cells Myeloid – CellFamily x tissue_origin.png"), units = "in", width = 5, height = 8, res = 300)

ggplot(Fam, aes(x = tissue_origin, y = Frequency, stratum = CellFamily, alluvium = CellFamily)) + 
    geom_hline(yintercept = c(0, 25, 50, 75, 100), color = 'gray90') +
    geom_alluvium(aes(fill = CellFamily), curve_type = "sigmoid", color = "gray80", lwd = .3, alpha = .3) + 
    geom_stratum(aes(fill = CellFamily), width = 7/16, lwd = .3, color = "black") + 
    theme_classic() + theme(axis.line = element_blank(), axis.ticks = element_blank()) +
    scale_y_continuous(expand = c(.01,0)) +
    scale_x_discrete(expand = c(.08, 0)) +
    scale_fill_manual(values = col) + 
    theme(legend.position = 'bottom', axis.title = element_blank())

dev.off()

## Heatmap top5 DEGs/CellFamily

In [ ]:
# Create the object to convert to Seurat object for DEGs
Myeloid.obs['CellFamily_tissue_origin'] = Myeloid.obs['CellFamily'].astype(str) + "_" + Myeloid.obs['tissue_origin'].astype(str)
Myeloid_raw = sc.AnnData(X = Myeloid.layers['raw_counts'],
                     obs = Myeloid.obs[['batch_id', 'tissue_origin', 'patient_ID', 'CellFamily', 'CellFamily_tissue_origin']],
                     var = Myeloid.var[['gene_ids']],
                     obsm = Myeloid.obsm)

outfilename = os.path.join(data_folder, "Myeloid_raw.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid_raw.write(outfilename)
print("Done!")

# Export metadata
Myeloid_raw.obs.to_excel('.../Analysis/Synchro/figures/Myeloid_metadata.xlsx')

In [ ]:
%%R
Convert(".../Analysis/Synchro/Data/Myeloid_raw.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
Myeloid_raw = LoadH5Seurat(".../Analysis/Synchro/Data/Myeloid_raw.h5seurat", meta.data = F)
meta = read.xlsx(".../Analysis/Synchro/figures/Myeloid_metadata.xlsx", rowNames = T)
Myeloid_raw@meta.data = meta
saveRDS(Myeloid_raw, ".../Analysis/Synchro/Data/Myeloid_raw.RDS")

In [ ]:
%%R
Idents(Myeloid_raw) = 'CellFamily_tissue_origin'
DEGs = FindAllMarkers(Myeloid_raw, logfc.threshold = 0, test.use = "negbinom", latent_vars = 'patient_ID')
write.xlsx(DEGs, "DEGs - Myeloid_raw – CellFamily_tissue_origin all vs all – negbinom.xlsx", rowNames = T)

In [ ]:
# Figure 1E
top5_CellFamily_tissue_origin = ["C15orf48", "CCL20", "IL1RN", "CD44", "SOD2",       #Inflammatory TAMs COLON
                                 "S100A12", "FCN1", "ZEB2", "VCAN", "SLC11A1",       #Inflammatory TAMs LIVER
                                 "RNASE1", "SELENOP", "MRC1", "CCL2", "LYVE1",       #Immunoregulatory TAMs COLON
                                 "HLA-DPA1", "HLA-DPB1", "IL1R2", "HLA-DRA", "RHOB", #Immunoregulatory TAMs LIVER
                                 "CPNE3", "SNX3", "CLEC9A", "CPVL", "HDAC9",         #cDC1 LIVER
                                 "LAMP3", "CCR7", "TXN", "FSCN1", "KDM2B",           #DC_LAMP3 COLON
                                 "BIRC3", "CCR7", "LAMP3", "FSCN1", "CD79A",         #DC_LAMP3 LIVER
                                 "GZMB", "FCHSD2", "IRF7", "SOX4", "IRF4",           #pDCs COLON
                                 "PTGDS", "GZMB", "IRF7", "CXCR3", "IRF4"]           #pDCs LIVER

order = ['Inflammatory TAMs_COLON', 'Inflammatory TAMs_LIVER', 'Immunoregulatory TAMs_COLON', 'Immunoregulatory TAMs_LIVER', 'cDC1_LIVER', 'DC_LAMP3_COLON', 'DC_LAMP3_LIVER', 'pDCs_COLON', 'pDCs_LIVER']

sc.pl.matrixplot(Myeloid, top5_CellFamily_tissue_origin, groupby = 'CellFamily_tissue_origin', swap_axes = False, standard_scale = 'var',
                 cmap = "Reds", 
                 categories_order = order,
                 dendrogram = False,
                 save = "Myeloid – top5_CellFamily_tissue_origin.png")

## Correlation matrix

In [ ]:
Myeloid_subset = Myeloid[~Myeloid.obs['CellFamily'].isin(['cDC1', 'DC_LAMP3', 'pDCs'])].copy()

In [ ]:
sc.tl.dendrogram(Myeloid_subset, 'Myeloid_site_short', use_rep = "X_scVI", cor_method = 'pearson')

In [ ]:
# Figure 4A
plt.rcParams['axes.grid'] = False
sc.pl.correlation_matrix(Myeloid_subset, 'Myeloid_site_short', cmap = 'RdGy_r',
                         save = 'Correlation Matrix Myeloid_no DCs.png')

## Density population plot

In [ ]:
sc.tl.embedding_density(Myeloid_subset, basis = 'umap', groupby = 'CellFamily')

In [ ]:
# Supplementary figure 4A right
Group = list(Myeloid_subset.obs['CellFamily'].unique())
for x in Group:
    sc.pl.embedding_density(Myeloid_subset, basis = 'umap', key = 'umap_density_CellFamily', color_map = 'Reds', group = f'{x}', bg_dotsize = 2, fg_dotsize = 20,
                        save = f'Myeloid total – DensityPlot population {x} – no DCs.png')

## Pseudotime analysis with Monocle2

In [ ]:
# Create the object to convert to Seurat object for Monocle2
Myeloid_subset = Myeloid[~Myeloid.obs['CellFamily'].isin(['cDC1', 'DC_LAMP3', 'pDCs'])].copy()
Myeloid_subset_norm = sc.AnnData(X = Myeloid_subset.X,
                                 obs = Myeloid_subset.obs[['batch_id', 'tissue_origin', 'patient_ID', 'CellFamily', 'Myeloid_site_short']],
                                 var = Myeloid_subset.var[['gene_ids']],
                                 obsm = Myeloid_subset.obsm)

outfilename = os.path.join(data_folder, "Myeloid_subset_norm.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid_subset_norm.write(outfilename)
print("Done!")

# Export metadata
Myeloid_subset_norm.obs.to_excel('.../Analysis/Synchro/figures/Myeloid_subset_metadata.xlsx')

In [ ]:
%%R
Convert(".../Analysis/Synchro/Data/Myeloid_subset_norm.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
Myeloid_subset_norm = LoadH5Seurat(".../Analysis/Synchro/Data/Myeloid_subset_norm.h5seurat", meta.data = F)
meta = read.xlsx(".../Analysis/Synchro/figures/Myeloid_subset_metadata.xlsx", rowNames = T)
Myeloid_subset_norm@meta.data = meta
saveRDS(Myeloid_subset_norm, ".../Analysis/Synchro/Data/Myeloid_subset_norm.RDS")

In [ ]:
%%R
Idents(Myeloid_subset_norm) = "Myeloid_site_short"
DimPlot(Myeloid_subset_norm, label = T)

### Running Monocle2

In [ ]:
%%R
library(monocle)

data = as(as.matrix(Myeloid_subset_norm@assays$RNA@data), 'sparseMatrix')
pd = new('AnnotatedDataFrame',data = Myeloid_subset_norm@meta.data)
fData = data.frame(gene_short_name = row.names(data), row.names = row.names(data))
fd = new('AnnotatedDataFrame', data = fData)
fd

In [ ]:
%%R
monocle_object = newCellDataSet(data,
                                phenoData = pd,
                                featureData = fd,
                                lowerDetectionLimit = 0.5,
                                expressionFamily = negbinomial.size())
monocle_object

In [ ]:
%%R
monocle_object = estimateSizeFactors(monocle_object)
monocle_object = estimateDispersions(monocle_object)

In [ ]:
%%R
monocle_object = detectGenes(monocle_object , min_expr = 0.1)

In [ ]:
# Ordering genes: import from HVG-scanpy function
sc.pp.highly_variable_genes(Myeloid_subset, n_top_genes = 4000, batch_key = 'tissue_origin', flavor = 'seurat_v3', layer = "raw_counts", subset = False)
HVG = Myeloid_subset.var[Myeloid_subset.var["highly_variable"]].index

In [ ]:
%%R -i HVG
ordering_genes = HVG
length(ordering_genes)

In [ ]:
%%R
monocle_object = setOrderingFilter(monocle_object, ordering_genes)
monocle_object = reduceDimension(monocle_object, max_components = 2, method = 'DDRTree')
monocle_object = orderCells(monocle_object)

In [ ]:
%%R
saveRDS(monocle_object, file = ".../Data/monocle_object.RDS")

### Plot

In [ ]:
%%R
# Figure 4B
png(file = file.path(".../figures/", 'monocle_object Pseudotime tissue_origin split.png'), units = "in", width = 1, height = 4, res = 1000)

color_tissue_origin = c('Blood' = '#1f77b4', 'Colon' = '#ff7f0e', 'Liver' = '#279e68')
plot_cell_trajectory(monocle_object, color_by = "tissue_origin") + facet_wrap(~tissue_origin, nrow = 1) + NoLegend() + scale_color_manual(values = color_tissue_origin) + NoLegend()

dev.off()

In [ ]:
%%R
# Figure 4C left
png(file = file.path(".../figures/", 'monocle_object Pseudotime values.png'), units = "in", width = 5.5, height = 4, res = 1000)

plot_cell_trajectory(monocle_object, color_by = "Pseudotime") + scale_color_viridis_c() + NoLegend()

dev.off()

In [ ]:
%%R
# Supplementary figure 4B
png(file = file.path(".../figures/", 'monocle_object Pseudotime Myeloid_site_short split.png'), units = "in", width = 25, height = 8, res = 1000)

color_clusters = c('B0'= '#7a49a5', 'B1'= '#7a49a5', 'B2'= '#7a49a5', 'B3'= '#7a49a5', 'B4'= '#7a49a5', 'B5'= '#0000ff',
                   'C0'= '#ffa52f', 'C2'= '#005659', 'C3'= '#0000dd', 'C4'= '#00fdcf', 'C5'= '#a17569', 'C6'= '#bcb6ff', 'C8'= '#bf03b8',
                   'L0'= '#790000', 'L1'= '#0774d8', 'L2'= '#fdf490', 'L3'= '#004b00', 'L4'= '#8e7900', 'L5'= '#ff7266', 'L6'= '#edb8b8', 'L7'= '#5d7e66', 'L8'= '#9ae4ff')
plot_cell_trajectory(monocle_object, color_by = "Myeloid_site_short") + facet_wrap(~Myeloid_site_short, nrow = nrow = 22) + scale_color_manual(values = color_clusters) + NoLegend()

dev.off()

### Export pseudotime values

In [ ]:
%%R
monocle_object$cellID = colnames(monocle_object)
df = data.frame(monocle_object$cellID)
rownames(df) = df$monocle_object.cellID

df$Batch_ID = monocle_object$batch_id
df$Myeloid_site_short = monocle_object$Myeloid_site_short
df$tissue_origin = monocle_object$tissue_origin
df$Pseudotime_Values = monocle_object$Pseudotime
df$State = monocle_object$State
df$monocle_object.cellID = NULL

write.xlsx(df, "../figures/monocle_object - Pseudotime_Values.xlsx", rowNames = T)

In [ ]:
# Figure 4C right
pseudotime = pd.read_excel('../figures/monocle_object - Pseudotime_Values.xlsx', index_col = 0)
Myeloid_subset.obs['pseudotime'] = pseudotime['Pseudotime_Values']

sc.pl.umap(Myeloid_subset, color = 'pseudotime', frameon = True, title = "Pseudotime", save = 'Myeloid – UMAP Pseudotime values – no DCs.png')

In [ ]:
%%R
# Figure 4D
library(ggridges)
library(dplyr)
library(tidyverse)

color_clusters = c(
  'cMono_CD14'= '#7a49a5', 'ncMono_FCGR3A'= '#0000ff',
  'C0'= '#ffa52f', 'C2'= '#005659', 'C3'= '#0000dd', 'C4'= '#00fdcf', 'C5'= '#a17569', 'C6'= '#bcb6ff', 'C8'= '#bf03b8',
  'L0'= '#790000', 'L1'= '#0774d8', 'L2'= '#fdf490', 'L3'= '#004b00', 'L4'= '#8e7900', 'L5'= '#ff7266', 'L6'= '#edb8b8', 'L7'= '#5d7e66', 'L8'= '#9ae4ff')

Ridge = read.xlsx(".../figures/monocle_object - Pseudotime_Values.xlsx")

png(file = file.path(".../figures/", 'Pseudotime values.png'), units = "in", width = 12, height = 6, res = 1000, bg = "white")

ggplot(Ridge, aes(y = fct_reorder(Myeloid_site_short, -Pseudotime_Values), x = Pseudotime_Values, fill = Myeloid_site_short)) +
    geom_density_ridges(quantile_lines = T, quantiles = 2, scale = 3.5, rel_min_height = .02, alpha = 0.9) +
    scale_y_discrete(expand = c(0, 0)) + scale_x_continuous(expand = c(0, 0)) + geom_vline(xintercept = 19.8, color = 'black', size = 1, linetype = "dashed") + 
    scale_fill_manual(values = color_clusters) + theme_ridges() + NoLegend()

dev.off()

## DEGs Colon vs Liver / Inflammatory & Immunoregulatory TAMs

In [ ]:
# Create the object to convert to Seurat object for DEGs
Myeloid.obs['CellFamily_tissue_origin'] = Myeloid.obs['CellFamily'].astype(str) + "_" + Myeloid.obs['tissue_origin'].astype(str)
Myeloid_raw = sc.AnnData(X = Myeloid.layers['raw_counts'],
                     obs = Myeloid.obs[['batch_id', 'tissue_origin', 'patient_ID', 'CellFamily', 'CellFamily_tissue_origin']],
                     var = Myeloid.var[['gene_ids']],
                     obsm = Myeloid.obsm)

outfilename = os.path.join(data_folder, "Myeloid_raw.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid_raw.write(outfilename)
print("Done!")

# Export metadata
Myeloid_raw.obs.to_excel('.../Analysis/Synchro/figures/Myeloid_metadata.xlsx')

In [ ]:
%%R
Convert(".../Analysis/Synchro/Data/Myeloid_raw.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
Myeloid_raw = LoadH5Seurat(".../Analysis/Synchro/Data/Myeloid_raw.h5seurat", meta.data = F)
meta = read.xlsx(".../Analysis/Synchro/figures/Myeloid_metadata.xlsx", rowNames = T)
Myeloid_raw@meta.data = meta
saveRDS(Myeloid_raw, ".../Analysis/Synchro/Data/Myeloid_raw.RDS")

In [ ]:
%%R
Idents(Myeloid_raw) = 'CellFamily_tissue_origin'
InflammatoryTAMs = subset(Myeloid_raw, idents = c("Inflammatory TAMs_LIVER", "Inflammatory TAMs_COLON")) 
DEGs = FindMarkers(InflammatoryTAMs, logfc.threshold = 0, test.use = "negbinom", ident.1 = "Inflammatory TAMs_LIVER", ident.2 = "Inflammatory TAMs_COLON", latent_vars = 'patient_ID')
write.xlsx(DEGs, ".../figures/Myeloid total – DEGs Inflammatory TAMs_LIVER (up) vs Inflammatory TAMs_COLON (down) – negbinom.xlsx", rowNames = T)

In [ ]:
%%R
Idents(Myeloid_raw) = 'CellFamily_tissue_origin'
ImmunoregulatoryTAMs = subset(Myeloid_raw, idents = c("Immunoregulatory TAMs_LIVER", "Immunoregulatory TAMs_COLON")) 
DEGs = FindMarkers(ImmunoregulatoryTAMs, logfc.threshold = 0, test.use = "negbinom", ident.1 = "Immunoregulatory TAMs_LIVER", ident.2 = "Immunoregulatory TAMs_COLON", latent_vars = 'patient_ID')
write.xlsx(DEGs, ".../figures/Myeloid total – DEGs Immunoregulatory TAMs_LIVER (up) vs Immunoregulatory TAMs_COLON (down) – negbinom.xlsx", rowNames = T)

In [ ]:
%%R
# Supplementary figure 5A
DEGs_Inflammatory = read.xlsx(".../figures/Myeloid total – DEGs Inflammatory TAMs_LIVER (up) vs Inflammatory TAMs_COLON (down) – negbinom.xlsx", rowNames = T)
DEGs_Immunoregulatory = read.xlsx(".../figures/Myeloid total – DEGs Immunoregulatory TAMs_LIVER (up) vs Immunoregulatory TAMs_COLON (down) – negbinom.xlsx", rowNames = T)

keyvals = rep('#5a5a5a', nrow(DEGs_Immunoregulatory))
names(keyvals) = rep('NS', nrow(DEGs_Immunoregulatory))

keyvals[which(abs(DEGs_Immunoregulatory$avg_log2FC) > 0.25 & DEGs_Immunoregulatory$p_val_adj > 0.05)] = '#808080'
names(keyvals)[which(abs(DEGs_Immunoregulatory$avg_log2FC) > 0.25 & DEGs_Immunoregulatory$p_val_adj > 0.05)] = 'Log2 – FC'

keyvals[which(abs(DEGs_Immunoregulatory$avg_log2FC) < 0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = '#a6a6a6'
names(keyvals)[which(abs(DEGs_Immunoregulatory$avg_log2FC)  < 0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = 'P-value'

keyvals[which(DEGs_Immunoregulatory$avg_log2FC < -0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = '#ff7f0e'
names(keyvals)[which(DEGs_Immunoregulatory$avg_log2FC  < -0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = 'Up-regulated Colon'

keyvals[which(DEGs_Immunoregulatory$avg_log2FC > 0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = '#279e68'
names(keyvals)[which(DEGs_Immunoregulatory$avg_log2FC > 0.25 & DEGs_Immunoregulatory$p_val_adj < 0.05)] = 'Up-regulated Liver'

png(file = file.path(".../figures/", 'DEGs_Immunoregulatory - Liver vs Colon.png'), units = "in", width = 9, height = 9, res = 1000)

EnhancedVolcano(DEGs_Immunoregulatory,
                lab = rownames(DEGs_Immunoregulatory),
                x = "avg_log2FC",
                y = "p_val_adj",
                axisLabSize = 15,
                subtitleLabSize = 20,
                captionLabSize = 20,
                pCutoff = 0.05,
                FCcutoff = 0.25,
                pointSize = 4,
                labSize = 6,
                colCustom = keyvals,
                labFace = 'italic',
                colAlpha = 0.75,
                gridlines.major = F,
                gridlines.minor = F,
                borderWidth = 1, borderColour = 'black',
                drawConnectors = T, widthConnectors = .5,
                max.overlaps = 35,
                boxedLabels = T)

dev.off()

## Reactome functional analysis

In [ ]:
%%R
# Figure 5A
Reactome_Inflammatory = read.xlsx('Reactome_Inflammatory.xlsx')
Reactome_Immunoregulatory = read.xlsx('Reactome_Immunoregulatory.xlsx')

Tissue_color = c("Colon" = '#ff7f0e', "Liver" = '#279e68')

# Inflammatory TAMs
label_data = Reactome_Inflammatory
number_of_bar = nrow(label_data)
angle = 90 - 360 * (label_data$id -0.5)/number_of_bar 
label_data$hjust = ifelse(angle < -90, 1, 0)
label_data$angle = ifelse(angle < -90, angle+180, angle)

base_data = Reactome_Inflammatory %>% 
  group_by(tissue_origin) %>% 
  summarize(start = min(id), end = max(id) - 2) %>% 
  rowwise() %>% 
  mutate(title = mean(c(start, end)))

grid_data = base_data
grid_data$end = grid_data$end[c(nrow(grid_data), 1:nrow(grid_data) -1)] +1
grid_data$start = grid_data$start -1

png(file = file.path(".../figures/", 'Reactome_Inflammatory circular bar plot.png'), units = "in", width = 20, height = 20, res = 1000, bg = "transparent")

ggplot(Reactome_Inflammatory, aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin)) +
    geom_bar(aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin), stat = "identity", alpha = 1) +

    geom_segment(data = grid_data, aes(x = end, y = 0, xend = start, yend = 0), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 2.5, xend = start, yend = 2.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 5, xend = start, yend = 5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 7.5, xend = start, yend = 7.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 10, xend = start, yend = 10), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 12.5, xend = start, yend = 12.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 15, xend = start, yend = 15), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
  
    annotate("text", x = rep(max(Reactome_Inflammatory$id), 7), y = c(0, 2.5, 5, 7.5, 10, 12.5, 15), label = c('0', '2.5', '5', '7.5', '10', '12.5', '15') , color = "black", size = 10, angle = 0, fontface = "italic", hjust = 1) +
  
    geom_bar(aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin), stat = "identity", alpha = 1) +
        scale_fill_manual(values = Tissue_color) +
        ylim(-4, 30) +
        theme_minimal() + 
        coord_polar(start = 0) +
        geom_text(data = label_data, aes(x = as.factor(id), y = minusLOG10_pValue + .5, label = Pathway, hjust = hjust), color = "black", fontface = "italic", alpha = 1, size = 10, angle = label_data$angle, inherit.aes = FALSE) + 

    theme(legend.position = "none", axis.text = element_blank(), axis.title = element_blank(), panel.grid = element_blank(), plot.margin = unit(rep(-2, 5), "cm"))

dev.off()

# Immunoregulatory TAMs
label_data = Reactome_Immunoregulatory
number_of_bar = nrow(label_data)
angle = 90 - 360 * (label_data$id -0.5)/number_of_bar 
label_data$hjust = ifelse(angle < -90, 1, 0)
label_data$angle = ifelse(angle < -90, angle+180, angle)

base_data = Reactome_Immunoregulatory %>% 
  group_by(tissue_origin) %>% 
  summarize(start = min(id), end = max(id) - 2) %>% 
  rowwise() %>% 
  mutate(title = mean(c(start, end)))

grid_data = base_data
grid_data$end = grid_data$end[c(nrow(grid_data), 1:nrow(grid_data) -1)] +1
grid_data$start = grid_data$start -1

png(file = file.path(".../figures/", 'Reactome_Immunoregulatory circular bar plot.png'), units = "in", width = 20, height = 20, res = 1000, bg = "transparent")

ggplot(Reactome_Immunoregulatory, aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin)) +
    geom_bar(aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin), stat = "identity", alpha = 1) +

    geom_segment(data = grid_data, aes(x = end, y = 0, xend = start, yend = 0), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 2.5, xend = start, yend = 2.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 5, xend = start, yend = 5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 7.5, xend = start, yend = 7.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 10, xend = start, yend = 10), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 12.5, xend = start, yend = 12.5), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
    geom_segment(data = grid_data, aes(x = end, y = 15, xend = start, yend = 15), colour = "grey", alpha = 1, size = 0.3, inherit.aes = FALSE ) +
  
    annotate("text", x = rep(max(Reactome_Immunoregulatory$id), 7), y = c(0, 2.5, 5, 7.5, 10, 12.5, 15), label = c('0', '2.5', '5', '7.5', '10', '12.5', '15') , color = "black", size = 10, angle = 0, fontface = "italic", hjust = 1) +
  
    geom_bar(aes(x = as.factor(id), y = minusLOG10_pValue, fill = tissue_origin), stat = "identity", alpha = 1) +
        scale_fill_manual(values = Tissue_color) +
        ylim(-4, 30) +
        theme_minimal() + 
        coord_polar(start = 0) +
        geom_text(data = label_data, aes(x = as.factor(id), y = minusLOG10_pValue + .5, label = Pathway, hjust = hjust), color = "black", fontface = "italic", alpha = 1, size = 10, angle = label_data$angle, inherit.aes = FALSE) + 

    theme(legend.position = "none", axis.text = element_blank(), axis.title = element_blank(), panel.grid = element_blank(), plot.margin = unit(rep(-2, 5), "cm"))

dev.off()

## Module score

In [ ]:
Myeloid_tissue_subset = Myeloid[~Myeloid.obs['CellFamily'].isin(['cDC1', 'DC_LAMP3', 'pDCs']) & ~Myeloid.obs['tissue_origin'].isin(['Blood'])].copy()
Myeloid_tissue_subset.obs['CellFamily_tissue_origin'] = Myeloid_tissue_subset.obs['CellFamily'].astype(str) + "_" + Myeloid_tissue_subset.obs['tissue_origin'].astype(str)

In [ ]:
# Figure 5B input data
ProInflammatory_cytokine = ['TNF', 'IL1A', 'IL1B', 'IL6', 'IL23A', 'CXCL9', 'CXCL10', 'CXCL11']
ImmuneSuppressive_cytokine = ['IL10', 'TGFB1', 'CCL2', 'CCL5', 'CCL17', 'CCL22']

sc.tl.score_genes(Myeloid_tissue_subset, ProInflammatory_cytokine, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'ProInflammatory_cytokine_Score', random_state = 0, copy = False)
sc.tl.score_genes(Myeloid_tissue_subset, ImmuneSuppressive_cytokine, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'ImmuneSuppressive_cytokine_Score', random_state = 0, copy = False)

Scores = Myeloid_tissue_subset.obs[['CellFamily_tissue_origin', 'ProInflammatory_cytokine_Score', 'ImmuneSuppressive_cytokine_Score']]

scaler = MinMaxScaler()
scores_scaled = scaler.fit_transform(Scores)
scores_scaled = pd.DataFrame(scores_scaled, columns = ['ProInflammatory_cytokine_Score_scaled', 'ImmuneSuppressive_cytokine_scaled'], index = Myeloid_tissue_subset.obs_names)
Scores = pd.concat([Myeloid_tissue_subset.obs[['CellFamily_tissue_origin']], Scores, scores_scaled], axis = 1)
Score.to_excel('figures/Myeloid_tissue_subset – Module scores cytokines x CellFamily_tissue_origin.xlsx', index = True)

In [ ]:
%%R
# Figure 5B Radar Chart
library(dplyr)

setwd(".../figures")
path = ".../figures"

RadarPlot = read.xlsx("Myeloid_tissue_subset – Module scores cytokines x CellFamily_tissue_origin.xlsx", rowNames = T)

ProInflammatory_cytokine_Score_scaled = RadarPlot %>%
    select(CellFamily_tissue_origin, ProInflammatory_cytokine_Score_scaled)

ImmuneSuppressive_cytokine_scaled = RadarPlot %>%
    select(CellFamily_tissue_origin, ImmuneSuppressive_cytokine_scaled)

Colors_scaled = c('ProInflammatory_cytokine_Score_scaled' = '#e8702a', 'ImmuneSuppressive_cytokine_scaled' = '#0c457d')
colors_in = alpha(Colors_scaled, 0.8)

png(file = file.path(path, 'ProInflammatory_cytokine_Score_scaled.png'), units = "in", width = 6, height = 6, res = 1000, bg = "white")

radarchart(ProInflammatory_cytokine_Score_scaled, 
            axistype = 0, axislabcol = 'grey10', 
            pcol = 'black', pfcol = alpha('#9e379f', 0.8), plwd = 1, plty = 1, pty = 32,
            cglcol = "grey50", cglty = 1,  cglwd = 1, 
            vlcex = 1)
legend(x = 1, y = 1, legend = rownames(ProInflammatory_cytokine_Score_scaled[-c(1,2),]), bty = "n", pch = 20 , col = colors_in, text.col = "grey", cex = 1.2, pt.cex = 3)

dev.off()

## Cell cycle analysis

In [ ]:
Myeloid_tissue_subset = Myeloid[~Myeloid.obs['CellFamily'].isin(['cDC1', 'DC_LAMP3', 'pDCs']) & ~Myeloid.obs['tissue_origin'].isin(['Blood'])].copy()

In [ ]:
cell_cycle_genes = [x.strip() for x in open('...figures/Cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in Myeloid_tissue_subset.var_names]
print(len(cell_cycle_genes))

In [ ]:
# Figure 5C input values
sc.tl.score_genes_cell_cycle(Myeloid_tissue_subset, s_genes = s_genes, g2m_genes = g2m_genes)
subset.obs.to_excel('.../figures/Cell cycle phases Myeloid_tissue_subset.xlsx', index = True)

In [ ]:
# Figure 5D input values
Liver = Myeloid_tissue_subset[Myeloid_tissue_subset.obs['tissue_origin'].isin(['Liver'])].copy()
sc.tl.score_genes(Liver, cell_cycle_genes, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'cell_cycle_genes_Score', random_state = 0, copy = False)

Scores = Liver.obs[['CellFamily', 'cell_cycle_genes_Score']]

scaler = MinMaxScaler()
scores_scaled = scaler.fit_transform(Scores)
scores_scaled = pd.DataFrame(scores_scaled, columns = ['cell_cycle_genes_Score'], index = CellFamily.obs_names)
Scores = pd.concat([Liver.obs[['CellFamily']], Scores, scores_scaled], axis = 1)
Score.to_excel('figures/Liver – cell_cycle_genes_Score x CellFamily.xlsx', index = True)

### DEGs cell cycle phases in liver

In [ ]:
# Create the object to convert to Seurat object for DEGs
ImmunoReg_Liver = Myeloid_tissue_subset[Myeloid_tissue_subset.obs['tissue_origin'].isin(['Liver']) & Myeloid_tissue_subset.obs['CellFamily'].isin(['Immunoregulatory TAMs'])].copy()
ImmunoReg_Liver_raw = sc.AnnData(X = ImmunoReg_Liver.layers['raw_counts'],
                     obs = ImmunoReg_Liver.obs[['batch_id', 'tissue_origin', 'patient_ID', 'phase']],
                     var = ImmunoReg_Liver.var[['gene_ids']],
                     obsm = ImmunoReg_Liver.obsm)

outfilename = os.path.join(data_folder, "ImmunoReg_Liver_raw.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
ImmunoReg_Liver_raw.write(outfilename)
print("Done!")

# Export metadata
ImmunoReg_Liver_raw.obs.to_excel('...figures/ImmunoReg_Liver_raw_metadata.xlsx')

In [ ]:
%%R
Convert(".../Data/ImmunoReg_Liver_raw.h5ad", dest = "h5seurat", assay = "RNA", overwrite = FALSE, verbose = TRUE)
ImmunoReg_Liver_raw = LoadH5Seurat(".../Analysis/Synchro/Data/ImmunoReg_Liver_raw.h5seurat", meta.data = F)
meta = read.xlsx(".../figures/ImmunoReg_Liver_raw_metadata.xlsx", rowNames = T)
ImmunoReg_Liver_raw@meta.data = meta
saveRDS(ImmunoReg_Liver_raw, ".../Analysis/Synchro/Data/ImmunoReg_Liver_raw.RDS")

In [ ]:
%%R
Idents(ImmunoReg_Liver_raw) = 'phase'
DEGs = FindAllMarkers(ImmunoReg_Liver_raw, logfc.threshold = 0, test.use = "negbinom", latent_vars = 'patient_ID')
write.xlsx(DEGs, ".../figures/ImmunoReg_Liver – DEGs phases – negbinom.xlsx", rowNames = T)

In [ ]:
# Supplementary figure 5B
Top10 = ['APOC1', 'FNDC3B',  'CTSD', 'TIMP1', 'PSAP', 'FCN1', 'WARS', 'RBM47', 'NEAT1','CD44',     #G1 phase
         'NASP', 'SLBP', 'MCM5', 'STMN1', 'H2AFZ', 'RPA2', 'PCNA', 'USP1', 'ATAD2', 'RFC2',        #S phase
         'CKS2', 'HMGB2', 'HIST1H4C', 'STMN1', 'ERO1B', 'SMC4', 'HMGN2', 'IER2', 'NUSAP1', 'TUBB'] #G2M phase

order = ['G1', 'S', 'G2M']

sc.pl.matrixplot(ImmunoReg_Liver, Top10, 'phase', standard_scale = 'var', swap_axes = True, dendrogram = False, categories_order = order, cmap = 'Reds',
                figsize = (6, 12), save = 'ImmunoReg_Liver – Heatmap top10 DEGs phases.png')

## Large & Small TAMs

In [ ]:
colors = {
    'L0': '#790000',
    'L1': '#0774d8',
    'L2': '#fdf490',
    'L3': '#004b00',
    'L4': '#8e7900',
    'L5': '#ff7266',
    'L6': '#edb8b8',
    'L7': '#5d7e66',
    'L8': '#9ae4ff'}

subset_tissue_liver = Myeloid[~Myeloid.obs['site'].isin(['Blood', 'Colon'])].copy()
subset_tissue_liver_macro = subset_tissue_liver[~subset_tissue_liver.obs['Myeloid_site_short'].isin(['L9', 'L11', 'L10'])].copy()

### PCA pseudo-bulking

In [ ]:
# Supplementary figure 5C top
L_TAM_genes = [x.strip() for x in open('.../figures/L_TAM_genes.txt')]   #281 genes

subset_tissue_liver_macro.var['gene_label'] = subset_tissue_liver_macro.var_names
subset_tissue_liver_macro_PCA_Large = subset_tissue_liver_macro[:, subset_tissue_liver_macro.var['gene_label'].isin(L_TAM_genes)].copy()

gene_ids = subset_tissue_liver_macro_PCA_Large.var.index.values
clusters = subset_tissue_liver_macro_PCA_Large.obs['Myeloid_site_short']
obs = subset_tissue_liver_macro_PCA_Large[:,gene_ids].X.toarray()
obs = pd.DataFrame(obs, columns = gene_ids, index = clusters)
average_obs = obs.groupby(level = 0).mean()
average_obs = average_obs

subset_tissue_liver_macro_PCA_Large = sc.AnnData(average_obs)

sc.tl.pca(subset_tissue_liver_macro_PCA_Large, svd_solver = 'arpack', use_highly_variable = False, n_comps = 3)
subset_tissue_liver_macro_PCA_Large.obs['Myeloid_site_short'] = subset_tissue_liver_macro_PCA_Large.obs_names

sc.pl.pca(subset_tissue_liver_macro_PCA_Large, color = 'Myeloid_site_short', palette = colors,
          components = 'all', projection = '3d', 
          frameon = False,  legend_loc = 'right margin',
          save = '.../figures/PCA subset_tissue_liver_macro_PCA_Large.png')

In [ ]:
# Supplementary figure 5C bottom
S_TAM_genes = [x.strip() for x in open('.../figures/S_TAM_genes.txt')]   #359 genes

gene_ids = subset_tissue_liver_macro_PCA_Small.var.index.values
clusters = subset_tissue_liver_macro_PCA_Small.obs['Myeloid_site_short']
obs = subset_tissue_liver_macro_PCA_Small[:,gene_ids].X.toarray()
obs = pd.DataFrame(obs, columns = gene_ids, index = clusters)
average_obs = obs.groupby(level = 0).mean()
average_obs = average_obs

subset_tissue_liver_macro_PCA_Small = sc.AnnData(average_obs)

sc.tl.pca(subset_tissue_liver_macro_PCA_Small, svd_solver = 'arpack', use_highly_variable = False, n_comps = 3)
subset_tissue_liver_macro_PCA_Small.obs['Myeloid_site_short'] = subset_tissue_liver_macro_PCA_Small.obs_names

sc.pl.pca(subset_tissue_liver_macro_PCA_Small, color = 'Myeloid_site_short', palette = colors,
          components = 'all', projection = '3d', 
          frameon = False,  legend_loc = 'right margin',
          save = '.../figures/PCA subset_tissue_liver_macro_PCA_Small.png')

In [ ]:
# Supplementary figure 5D input data
df_loadings_Large = pd.DataFrame(subset_tissue_liver_macro_PCA_Large.varm['PCs'], index = subset_tissue_liver_macro_PCA_Large.var_names)
df_loadings_Small = pd.DataFrame(subset_tissue_liver_macro_PCA_Small.varm['PCs'], index = subset_tissue_liver_macro_PCA_Small.var_names)

df_loadings_Large[0].sort_values().to_csv('.../figures/L-TAMs – subset_tissue_liver_macro_PCA_Large PCA loadings.csv')
df_loadings_Small[0].sort_values().to_csv('.../figures/S-TAMs – subset_tissue_liver_macro_PCA_Small PCA loadings.csv')

In [ ]:
# Supplementary figure 5D
PC1_20_Large = ['C1QA', 'C1QC', 'C1QB', 'FCGR3A', 'FMNL2', 'PDK4', 'PLTP', 'SELENOP', 'LGMN', 'VSIG4', 'LIPA', 'APOE', 'SLCO2B1', 'FRMD4A', 'SLC40A1', 'APOC1', 'SPATS2L', 'FOLR2', 'FCHO2', 'FRMD4B']
PC1_20_Small = ['S100A8', 'S100A9', 'VCAN', 'FCN1', 'THBS1', 'S100A6', 'SLC11A1', 'ANPEP', 'AQP9', 'S100A4', 'LYZ', 'EREG', 'MAP4K4', 'S100A12', 'TSPO', 'SLC25A37', 'PLCB1', 'CD52', 'CD300E', 'APOBEC3A']

sc.tl.score_genes(subset_tissue_liver_macro, PC1_20_Large, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'L_TAM_Score_Top20', random_state = 0, copy = False)
sc.tl.score_genes(subset_tissue_liver_macro, PC1_20_Small, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'S_TAM_Score_Top20', random_state = 0, copy = False)

PC1_genes = ['C1QA', 'C1QC', 'C1QB', 'FCGR3A', 'FMNL2', 'PDK4', 'PLTP', 'SELENOP', 'LGMN', 'VSIG4', 'LIPA', 'APOE', 'SLCO2B1', 'FRMD4A', 'SLC40A1', 'APOC1', 'SPATS2L', 'FOLR2', 'FCHO2', 'FRMD4B',
             'S100A8', 'S100A9', 'VCAN', 'FCN1', 'THBS1', 'S100A6', 'SLC11A1', 'ANPEP', 'AQP9', 'S100A4', 'LYZ', 'EREG', 'MAP4K4', 'S100A12', 'TSPO', 'SLC25A37', 'PLCB1', 'CD52', 'CD300E', 'APOBEC3A']

order = ['L3', 'L8', 'L4', 'L6', 'L0', 'L1', 'L2', 'L5', 'L7']  

sc.pl.matrixplot(subset_tissue_liver_macro, PC1_genes, 'Myeloid_site_short',
                 standard_scale = 'var',  cmap = 'Reds',
                 swap_axes = True, categories_order = order,
                 save = 'PC1_genes Small and Large – Liver Macrophages.png')

sc.pl.matrixplot(subset_tissue_liver_macro, ['L_TAM_Score_Top20', 'S_TAM_Score_Top20'], 'Myeloid_site_short',
                 standard_scale = 'var',  cmap = 'Reds',
                 swap_axes = True, categories_order = order,
                 save = 'PC1_genes_scores Small and Large – Liver Macrophages.png')

### Cell cycle proliferation

In [ ]:
S_L_TAM = subset_tissue_liver_macro[subset_tissue_liver_macro.obs['Myeloid_site_short'].isin(['L2', 'L5', 'L7', 'L3', 'L8'])].copy()
S_L_TAM.obs['S_L_TAM'] = S_L_TAM.obs['Myeloid_site_short'].map({
    'L2': 'LTAM',
    'L5': 'LTAM',
    'L7': 'LTAM',
    'L3': 'STAM',
    'L8': 'STAM'})

top10_SphaseG2Mphase = ['NASP', 'SLBP', 'MCM5', 'STMN1', 'H2AFZ', 'RPA2', 'PCNA', 'USP1', 'ATAD2', 'RFC2', 'CKS2', 'HMGB2', 'HIST1H4C', 'STMN1', 'ERO1B', 'SMC4', 'HMGN2', 'IER2', 'NUSAP1', 'TUBB']
sc.tl.score_genes(S_L_TAM, top10_SphaseG2Mphase, ctrl_size = 50, gene_pool = None, n_bins = 25, score_name = 'top10_SphaseG2Mphase_Score', random_state = 0, copy = False)

Score = S_L_TAM.obs[['S_L_TAM', 'top10_SphaseG2Mphase_Score']]
Score.to_excel('.../figures/S_L_TAMs – Module score values top10_SphaseG2Mphase.xlsx', index = True)